Attempting to use Sørensen–Dice coefficient as means of grouping similar data entries
- https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
- Provides a fractional score between 0 and 1
- Does not obey triangle equality, but that does not seem like a downside at this point

Following implementation is shamelessly copied from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Dice%27s_coefficient#Python

In [1]:
def dice_coefficient(a,b):
    if not len(a) or not len(b): return 0.0
    """ quick case for true duplicates """
    if a == b: return 1.0
    """ if a != b, and a or b are single chars, then they can't possibly match """
    if len(a) == 1 or len(b) == 1: return 0.0
    
    """ use python list comprehension, preferred over list.append() """
    a_bigram_list = [a[i:i+2] for i in range(len(a)-1)]
    b_bigram_list = [b[i:i+2] for i in range(len(b)-1)]
    
    a_bigram_list.sort()
    b_bigram_list.sort()
    
    # assignments to save function calls
    lena = len(a_bigram_list)
    lenb = len(b_bigram_list)
    # initialize match counters
    matches = i = j = 0
    while (i < lena and j < lenb):
        if a_bigram_list[i] == b_bigram_list[j]:
            matches += 2
            i += 1
            j += 1
        elif a_bigram_list[i] < b_bigram_list[j]:
            i += 1
        else:
            j += 1
    
    score = float(matches)/float(lena + lenb)
    return score

Pseudocode

In [7]:
"""
define dice score threshold
load dataframe from Excel sheet
define list of columns that can be excluded from analysis
make a new column which is every included column concatenated
    defines the strings used in the analysis, will refer to them as entries
make a new column called sort index
    each "cluster" will share an integer
    assign initial value of 0
sort_index = 1
for every row
    if row[sort index] = 0
        row[sort index] = sort_index
        for every other row (probably implement as another for every row)
            if other row[sort index] = 0
                dice score (entry, other entry)
                if threshold < dice score
                    other row[sort index] = sort index
        sort index ++
"""

'\nload dataframe from Excel sheet\ndefine list of columns that can be excluded from analysis\nmake a new column which is every included column concatenated\n    defines the strings used in the analysis, will refer to them as entries\nmake a new column called sort index\n    each "cluster" will share an integer\n    assign initial value of 0\nsort_index = 1\nfor every row\n    if row[sort index] = 0\n        row[sort index] = sort_index\n        for every other row (probably implement as another for every row)\n            if row[sort index] = 0\n                dice score (entry, other entry)\n        sort index ++\n'

# 2022-10-25
Finding a good threshhold score

In [2]:
path = "F:/EXCEL Sheets for ADNAN to MERGE 2022/merged.xlsx"
import pandas as pd

In [7]:
df = pd.read_excel(path)
df.shape

(9504, 41)